In [ ]:
import os

if not os.getcwd().endswith("src"):
    %cd ..
%pwd

/home/xqz-u/master/FACT/FACT/src


'/home/xqz-u/master/FACT/FACT/src'

In [ ]:
import itertools as it
from copy import deepcopy
import logging
import os
import pprint
from typing import Dict, Sequence

import implicit
from implicit import evaluation
import numpy as np
import pandas as pd
import scipy
from scipy import sparse

import config
import constants
import utils

logger = logging.getLogger()
utils.setup_root_logging()

print(f"Implcit using GPU: {implicit.gpu.HAS_CUDA}")

/home/xqz-u/py_envs/FACT/lib/python3.10/site-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: CUDA driver version is insufficient for CUDA runtime version (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


Implcit using GPU: False


In [ ]:
# steps:
#     1. transform into full user-item preference matrix
#     2. keep only top-2500 most listened artists
#     3. pre-process raw counts with log transforms (is it just taking the log?)
#     4. split into 70/10/20 train/val/test sets, save the seeds used
#     5. use Implicit library to fit a matrix factorization, using
#        grid-search on hyperparms defined in appendix C.2
#     6. generalize to MovieLens dataset, gpu etc.

In [ ]:
user_item_df = pd.read_csv(config.LASTFM_DIR / "user_artists.dat", sep="\t")
user_item_df = user_item_df.rename(columns={"userID": "user", "artistID": "item"})
user_item_df = user_item_df.pivot(index="user", columns="item", values="weight").fillna(0)
user_item_df

item,1,2,3,4,5,6,7,8,9,10,...,18736,18737,18738,18739,18740,18741,18742,18743,18744,18745
user,,,,,,,,,,,,,,,,,,,,,
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2095,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2096,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2097,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# keep only top k artists in each row (user)
k = 2500
top_k_artists = user_item_df.sum(axis=0).sort_values(ascending=False).index[:k]
# drops only columns of user_item_df which do not appear in top_k_artists
user_item_df = user_item_df[user_item_df.columns.intersection(top_k_artists)]
assert (user_item_df.columns == top_k_artists.sort_values()).all()
user_item_df

# NOTE
# The completed preference matrix has columns indexed from 0 to 2500, which
# should be mapped to top_k_artists.sort_values() if the actual item id is
# wanted, which should not be the case for our purposes

item,2,6,7,8,9,10,12,15,18,19,...,18125,18126,18127,18205,18206,18434,18435,18558,18559,18575
user,,,,,,,,,,,,,,,,,,,,,
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2095,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2096,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2097,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# log-trasform
user_item_df = np.log(user_item_df, where=user_item_df > 0)
user_item_df

item,2,6,7,8,9,10,12,15,18,19,...,18125,18126,18127,18205,18206,18434,18435,18558,18559,18575
user,,,,,,,,,,,,,,,,,,,,,
2,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2095,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2096,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2097,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# implicit wants sparse matrices (user, item), the docs say (item, user) but they are outdated,
# look at their source code instead
user_item_csr = scipy.sparse.csr_matrix(user_item_df.values)
user_item_csr

<1892x2500 sparse matrix of type '<class 'numpy.float64'>'
	with 67009 stored elements in Compressed Sparse Row format>

In [ ]:
# split into 0.7 train 0.2 val 0.1 test
seed = 42
rng = np.random.default_rng(seed=seed)

train_csr, tmp_csr = evaluation.train_test_split(user_item_csr, train_percentage=0.7, random_state=seed)
val_csr, test_csr = evaluation.train_test_split(tmp_csr, train_percentage=2/3, random_state=seed)
train_csr, val_csr

(<1892x2500 sparse matrix of type '<class 'numpy.float64'>'
 	with 47004 stored elements in Compressed Sparse Row format>,
 <1892x2500 sparse matrix of type '<class 'numpy.float64'>'
 	with 13362 stored elements in Compressed Sparse Row format>)

In [ ]:
def recommender_grid_search(
    train_mat: sparse.csr_matrix,
    valid_mat: sparse.csr_matrix,
    best_model_paths: Sequence[str],
    metric: str = "map",
    **hyperparams: Dict[str, Sequence],
) -> implicit.als.AlternatingLeastSquares:
    logger.info("Hyperparameters in grid search:")
    logger.info(pprint.pformat(hyperparams))
    hyperparams_flat = list(it.product(*hyperparams.values()))

    best_model_path, best_hparams_path = best_model_paths
    best_score, best_model, best_hparams = -1.0, None, None

    for i, hparams in enumerate(hyperparams_flat):
        model = implicit.als.AlternatingLeastSquares(
            **dict(zip(hyperparams.keys(), hparams))
        )
        model.fit(train_mat)
        score = evaluation.ranking_metrics_at_k(model, train_mat, valid_mat)[metric]
        if score > best_score:
            logger.info(
                "%d: Best model found! Old %s: %f new %s: %f hparams: %s",
                i,
                metric,
                best_score,
                metric,
                score,
                hparams,
            )
            best_score = score
            best_model = deepcopy(model)
            best_hparams = hparams

    best_model.save(best_model_path)
    logger.debug("Saved best model to %s", best_model_path)
    with open(best_hparams_path, "w") as fd:
        fd.write("factor,regularizer,alpha,metric,score\n")
        fd.write(f"{','.join(list(map(str, best_hparams + (metric, best_score))))}\n")
    logger.debug("Saved best model hyperparams to %s", best_hparams_path)
    return best_model

In [ ]:
ground_truth_paths = [
    config.MODELS_DIR / "model_lastfm_ground_truth.npz",
    config.MODELS_DIR / "hparams_lastfm_ground_truth.npz",
]

# best_model = recommender_grid_search(
#     train_csr,
#     val_csr,
#     ground_truth_paths,
#     **constants.ground_truth_hparams,
# )
best_model = implicit.cpu.als.AlternatingLeastSquares.load(ground_truth_paths[0])

In [ ]:
# low-rank matrix completion
# ground_truth = best_model.user_factors.to_numpy() @ best_model.item_factors.to_numpy().T
# save the ground truths
# savepath = config.MODELS_DIR / "ground_truth_lastfm.npy"
# np.save(savepath, ground_truth)
ground_truth = np.load(config.MODELS_DIR / "ground_truth_lastfm.npy")

In [ ]:
# sampled_items_proportion = int(0.2 * ground_truth.shape[1])
# sampled_preferences = np.array([rng.choice(ground_truth.shape[1], size=sampled_items_proportion, replace=False) for i in range(ground_truth.shape[0])])
# print(sampled_preferences.shape)
# ground_truth_masked = np.zeros_like(ground_truth)
# np.put_along_axis(ground_truth_masked, sampled_preferences, np.take_along_axis(ground_truth, sampled_preferences, axis=1), axis=1)
# print(ground_truth_masked)
# NOTE below samples the same indices for each row hence not suitable
# x = np.repeat(np.arange(ground_truth.shape[1])[None, :], ground_truth.shape[0], axis=0)
# print(x[:, rng.choice(x.shape[1], int(0.2*x.shape[1]), replace=False)])

In [ ]:
# we mask 80% of the ground truth data because in section 5.1 they say:
# the simulated recommender system estimates relevance scores using low-rank
# matrix completion (Bell and Sejnowski 1995) on a training sample of 20% of
# the ground truth preferences
indices = [(i, j) for i in range(ground_truth.shape[0]) for j in range(ground_truth.shape[1])]
sample = rng.choice(indices, size=int(0.2 * len(indices)), replace=False)
ground_truth_masked = np.zeros_like(ground_truth)
ground_truth_masked[sample[:, 0], sample[:, 1]] = ground_truth[sample[:, 0], sample[:, 1]]

ground_truth_masked_sparse = scipy.sparse.csr_matrix(ground_truth_masked)
rec_train_csr, tmp_csr = evaluation.train_test_split(ground_truth_masked_sparse, train_percentage=0.7, random_state=seed)
rec_val_csr, rec_test_csr = evaluation.train_test_split(tmp_csr, train_percentage=2/3, random_state=seed)
rec_train_csr, rec_val_csr

(<1892x2500 sparse matrix of type '<class 'numpy.float32'>'
 	with 636565 stored elements in Compressed Sparse Row format>,
 <1892x2500 sparse matrix of type '<class 'numpy.float32'>'
 	with 106499 stored elements in Compressed Sparse Row format>)

In [ ]:
recommender_paths = [
    config.MODELS_DIR / "model_lastfm.npz",
    config.MODELS_DIR / "hparams_lastfm.txt",
]

# best_recommender = recommender_grid_search(
#     rec_train_csr, rec_val_csr, recommender_paths, **constants.recommender_hparams
# )
best_recommender = implicit.cpu.als.AlternatingLeastSquares.load(recommender_paths[0])

In [ ]:
# NOTE works with both the full user-item preferences matrix and a single user's preferences,
# just add a batch dimension in the latter case
def user_policy(
    user_preferences: np.ndarray, temperature: float, rng: np.random.Generator
) -> np.ndarray:
    # apply softmax with inverse temperature
    # formula: https://i.stack.imgur.com/HYyQT.jpg
    recommendation_probs = scipy.special.softmax(user_preferences / temperature, axis=1)
    # retrieve the indexes of the most recommended item for each user according to the
    # softmax probabilities
    recommended_items_indexes = np.array(
        [
            rng.choice(user_preferences.shape[1], replace=False, p=user_probs)
            for user_probs in recommendation_probs
        ]
    )
    # recommended_items_indexes = np.argmax(recommendation_probs, axis=1, keepdims=True)
    # generate binary recommendations matrix
    recommended_items = np.zeros_like(user_preferences)
    np.put_along_axis(recommended_items, recommended_items_indexes[:, None], 1, axis=1)
    assert (np.where(recommended_items > 0)[1] == recommended_items_indexes).all()
    return recommended_items


# reconstruct user preferences matrix with the recommender system factors
temperature = 5
user_preferences = best_recommender.user_factors @ best_recommender.item_factors.T
user_recommendations = user_policy(user_preferences, temperature, rng)

In [ ]:
# generate binary rewards using bernoulli distribution
# bernoulli distribution is a special case of binomial distribution
# it is the probability mass function of a binary random variable
# i.e. with outcomes k=0, k=1  - wich relates to our recommended items elements values
# the bernoulli distribution has a probability mass function of:
# f(x)=p if k=1
# f(x)=1-p if k=0
# p is the expected value of the bernoulli distribution function
# we will use p = corresponding ground truth value as per paper instructions
def generate_rewards(
    true_preferences: np.ndarray, temperature: float, rng: np.random.Generator
) -> np.ndarray:
    true_probs = scipy.special.softmax(ground_truth / temperature, axis=1)
    # binomial with n=1 == bernoulli distribution
    return rng.binomial(1, true_probs)


def single_bernoulli(recommendation: np.ndarray, ground_truth: np.ndarray) -> int:
    # using compact definition f(x)=(p^k)*((1-p)^(1-k))
    probability = (ground_truth**recommendation)*((1-ground_truth)**(1-recommendation))
    # we now generate a binary reward which takes value 1 with the probability just computed and 0 otherwise
    # random.random returns a random number between 0 and 1, random.random() is less than p with probability p 
    reward = rng.random() < probability
    return int(reward)

# TODO clarify with teacher whether normalization on the ground truth
# needs to be done or not (perhaps original range is not suitable)

In [ ]:
ground_truth

array([[ 0.04816863,  0.01495649, -0.21136646, ...,  0.00216692,
         0.        ,  0.        ],
       [ 0.0012885 , -0.00414699,  0.00849485, ...,  0.00153843,
         0.        ,  0.        ],
       [ 0.0677122 , -0.02562742, -0.08222081, ...,  0.0084919 ,
         0.        ,  0.        ],
       ...,
       [-0.02334931, -0.00854431,  0.20373574, ...,  0.02270191,
         0.        ,  0.        ],
       [ 0.03085939, -0.00660964, -0.05778283, ...,  0.00844021,
         0.        ,  0.        ],
       [ 0.01000556,  0.09633859, -0.04463174, ..., -0.00779628,
         0.        ,  0.        ]], dtype=float32)

In [ ]:
true_probs = scipy.special.softmax(ground_truth / temperature, axis=1)
true_probs[0]

array([0.00040036, 0.00039771, 0.00038011, ..., 0.0003967 , 0.00039653,
       0.00039653], dtype=float32)

In [ ]:
recommendation_probs = scipy.special.softmax(user_preferences / temperature, axis=1)
recommended_items_indexes = np.array(
    [
        rng.choice(user_preferences.shape[1], replace=False, p=user_probs)
        for user_probs in recommendation_probs
    ]
)

In [ ]:
u0_rec = recommended_items_indexes[0]
print(f"user 0 recommendation: {u0_rec} corresponding preference: {recommendation_probs[0, u0_rec]}")
print(f"true preference at recommedation {u0_rec} is {true_probs[0, u0_rec]}")

u0_true_recs = rng.binomial(1, true_probs[0])
possible_rewards_ids = np.argwhere(u0_true_recs > 0)
if possible_rewards_ids.size > 0:
    u0_true_probs = np.take_along_axis(true_probs[0][:, None], possible_rewards_ids, axis=0).squeeze()
    print(f"true recommendations are {possible_rewards_ids.squeeze()}")
    print(f"with probabilities {u0_true_probs} (bernoulli)")
else:
    print("nothing can give pos reward for user 0")
reward = u0_true_recs[u0_rec]
print(f"reward -> true_recommendations[0, {u0_rec}] = {u0_true_recs[u0_rec]}")

user 0 recommendation: 1400 corresponding preference: 0.00039957856643013656
true preference at recommedation 1400 is 0.0004044487723149359
true recommendations are [ 468 2319 2445]
with probabilities [0.00039347 0.00040037 0.00040222] (bernoulli)
reward -> true_recommendations[0, 1400] = 0


In [ ]:
true_recs = rng.binomial(1, true_probs)
rewards = np.take_along_axis(true_recs, recommended_items_indexes[:, None], axis=1)
print(f"indexes of +1 rewards: {np.argwhere(rewards > 0)}")

indexes of +1 rewards: []


In [ ]:
# QUESTION how do we reduce the reward to just one number?
# do we say, if the recommender recommended something which has 1 in
# true_rewards_mask, then reward=1, else reward=0? Is this the
# correct way to generate rewards *at all*?
true_rewards_mask = generate_rewards(ground_truth, temperature, rng)
print(true_rewards_mask)
# sum over columns to get number of true recommendations indices for
# each user; then check how many of them are nonzero out of 1892
print(np.argwhere(true_rewards_mask.sum(1)).squeeze().shape)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(1212,)


In [ ]:
# moreover, there is a problem with this interpretation:
# a user could have 0 true recommendations...
user_recommendations[0].nonzero(), true_rewards_mask[0].nonzero()

((array([1947]),), (array([], dtype=int64),))

In [ ]:
vectorized_bernoulli = np.vectorize(single_bernoulli)
binary_rewards = vectorized_bernoulli(user_recommendations, ground_truth)
print(binary_rewards.sum(axis=1).sum())

4535748


In [ ]:
bandit_1 = np.concatenate(([0.6], [0.3] * 9))
print(bandit_1)
rng.binomial(1, bandit_1)

[0.6 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3]


array([0, 1, 0, 1, 0, 0, 0, 1, 1, 0])